[View in Colaboratory](https://colab.research.google.com/github/Smartappli/Gateway/blob/master/2011.ipynb)

In [1]:
!/opt/bin/nvidia-smi

Sun Sep 23 18:39:48 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Ci-dessous, le premier classifieur 

Variante pour l'exercice ci-dessus : essayer avec Ronaldo plutôt que Messi (/10)

---



In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
import numpy as np
np.random.seed(42)

In [0]:
from keras.datasets import mnist

In [5]:
(x_train, y_train), (x_test, y_test)=mnist.load_data()

11493376/11490434 [==============================] - 8s 1us/step


In [6]:
print (x_train.shape)
print (y_train.shape)

(60000, 28, 28)
(60000,)


In [0]:
x_train=x_train.reshape (x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test=x_test.reshape (x_test.shape[0], x_test.shape[1]*x_test.shape[2])

In [0]:
x_train=x_train.astype('float64')/255
x_test=x_test.astype('float64')/255

In [9]:
print (y_test)

[7 2 1 ... 4 5 6]


In [10]:
nb_classes=10
y_train=np_utils.to_categorical(y_train,nb_classes)
y_test=np_utils.to_categorical(y_test,nb_classes)
print (y_test)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
print (x_train.shape)
from sklearn.model_selection import train_test_split
# take validation set
trainX, validX, trainY, validY=train_test_split(x_train, y_train, test_size=0.33, random_state=42) # same random_state than seed above
print (trainX.shape)
print (validX.shape)
print (trainY.shape)
print (validY.shape)



(60000, 784)
(40200, 784)
(19800, 784)
(40200, 10)
(19800, 10)


In [0]:
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
model=Sequential()
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]
# first layer, always precise input_dim
model.add (Dense(100,input_dim=trainX.shape[1], activation='elu'))
# le dropout permet de "désactiver" aléatoirement un certain pourcentage de neurones de la couche précédente à chaque epoch, l'objectif étant une meilleure généralization
model.add (Dropout (0.20))
model.add (Dense(100, activation='relu'))
model.add (Dropout (0.15))
model.add (Dense(100, activation='relu'))
model.add (Dropout (0.10))
# last layer, 10 neurons (obligé)
model.add(Dense(nb_classes, activation='softmax'))


In [0]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]
# early stopping permet de s'arrêter dès que l'erreur validation ne diminue plus (après "patience" itérations)
import matplotlib.pyplot as plt 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print (model.summary())
history=model.fit(trainX, trainY,validation_data=(validX, validY), callbacks=callbacks_list,  epochs=100) #validation_data=(validX, validY), callbacks=callbacks_list
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

train_accuracy=model.evaluate(trainX, trainY)[1]
print ("Train accuracy = ", train_accuracy)
valid_accuracy=model.evaluate(validX, validY)[1]
print ("Valid accuracy = ", valid_accuracy)
test_accuracy=model.evaluate(x_test, y_test)[1]
print ("Test accuracy = ", test_accuracy)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               78500     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
Total para

In [14]:
data_test=x_test[0,:]
print (y_test[0])
data_test=data_test.reshape(1,data_test.shape[0])
print (data_test.shape)
model.predict(data_test)

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(1, 784)


array([[5.2269405e-07, 1.2915089e-07, 1.5542646e-05, 1.4325685e-05,
        4.6842352e-10, 1.0775715e-06, 1.7818179e-11, 9.9994969e-01,
        1.4881658e-07, 1.8548788e-05]], dtype=float32)

In [0]:
#perspectives : CNN, LSTM (should give better results)